<a href="https://colab.research.google.com/github/Ali-mohammadi-design/Idea_Modifier_LLM_Advisor/blob/main/Idea_Modifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


**Definign Problem and the Idea**

In [112]:
problem="I need to buy a house"
idea='Get a loan from bank'
problem_idea=f"The problem is {problem}. \n the idea is {idea}"

**Entering Open AI API Key**

In [113]:
import os
openAI_API_Key=os.environ['openAI_API_Key']
llm_config = {"model": "gpt-3.5-turbo",  "api_key":openAI_API_Key }

**Simple Idea Modifier**

In [114]:
import autogen

In [115]:
Idea_modifier_1 = autogen.AssistantAgent(
    name="Idea_modifier",
    system_message='''
    As an AI assistant, you will receive a problem and an idea to solve it.
    You may also get some criticism about the idea.
    In such cases, you need to assess the criticisms.
    If you believe the criticisms are valid, then you should modify the idea and clearly state the new idea.
    However, if you think the criticisms are insignificant, you should explain why the idea is still beneficial.''',
    llm_config=llm_config,
)

In [116]:
reply = Idea_modifier_1.generate_reply(messages=[{"content": problem_idea, "role": "user"}])

In [117]:
print(reply)

Criticism: Getting a loan means having debt and paying interest on it in the long run, which can be financially burdensome.

Given the high cost of purchasing a house, getting a loan from a bank is a common and practical solution for many people. Here's why it can still be beneficial:

1. **Ownership**: Despite having a loan, you will still own the house. Over time, as you pay off the loan, you build equity in the property, which can be a valuable asset.

2. **Tax Benefits**: Depending on where you live, mortgage interest payments may be tax-deductible, providing some financial relief.

3. **Investment**: Real estate is generally considered a good long-term investment. The value of the property may appreciate over time, potentially exceeding the cost of the loan and interest payments.

4. **Liquidity**: Unlike other forms of investment, a house can also serve as a place to live, providing both shelter and potential financial return.

While it's essential to carefully consider the terms

**Adding Critics**

In [124]:
critic_1 = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message='''As a critic, your role is to thoroughly evaluate the idea presented to you.
     Your task is to identify and articulate the weaknesses of the idea and provide a justification for each weakness you mention.''',



)

**To get advice for general problems and ideas**

In [125]:
res = Idea_modifier_1.initiate_chat(
    recipient=critic_1,
    message=problem_idea,
    max_turns=3,
    summary_method="last_msg"
)

Idea_modifier (to Critic):

The problem is I need to buy a house. 
 the idea is Get a loan from bank

--------------------------------------------------------------------------------
Critic (to Idea_modifier):

There are several weaknesses in the idea of getting a loan from the bank to buy a house:

1. Interest rates: One of the primary concerns with taking out a loan from the bank is the interest rates. Interest rates can fluctuate and, if not locked in at a favorable rate, can end up costing significantly more over the term of the loan. This can ultimately make the purchase of the house more expensive in the long run.

2. Debt burden: Taking out a loan to buy a house means taking on debt. Depending on the terms of the loan and the individual's financial situation, the debt burden may become overwhelming and impact the individual's financial stability in the long term. This can lead to stress and financial difficulties if the individual is unable to keep up with the loan payments.

3.

**Nested chat**

In [98]:
Idea_modifier = autogen.AssistantAgent(
    name="Idea_modifier",
    system_message='''
    As an AI assistant, you will receive a problem and an idea to solve it.
    You may also get some criticism about the idea.
    In such cases, you need to assess the criticisms.
    If you believe the criticisms are valid, then you should modify the idea and clearly state the new idea.
    However, if you think the criticisms are insignificant, you should explain why the idea is still beneficial.''',
    llm_config=llm_config,
)

In [99]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message='''As a critic, your role is to thoroughly evaluate the idea presented to you.
     Your task is to identify and articulate the weaknesses of the idea and provide a justification for each weakness you mention.'''

                ,
)

In [100]:
Feasibility_reviewer = autogen.AssistantAgent(
    name="Feasibility_reviewer",
    llm_config=llm_config,
    system_message='''You are a critic. You review the idea from its feasibility aspects.
Technical Feasibility: Can the idea be technically implemented with current resources and technology?
Operational Feasibility: Can the idea be integrated into existing operations or processes?
Financial Feasibility: Is the idea affordable within budget constraints? What are the cost implications?
''',
)

In [101]:
innovation_reviewer = autogen.AssistantAgent(
    name="innovation_reviewer",
    llm_config=llm_config,
    system_message='''You are a critic. You review the idea from its innovation aspects.
Originality: How novel or unique is the idea?
Creative Potential: Does the idea have potential for inspiring further innovation or creative solutions?
''',
)

In [102]:
impact_reviewer = autogen.AssistantAgent(
    name="impact Reviewer",
    llm_config=llm_config,
    system_message='''You are a critic. You review the idea from its Impact aspects.
Effectiveness: Will the idea effectively solve the problem it’s intended to address?
Reach: How broad is the potential impact of the idea? Will it benefit many people or just a few?
Sustainability: Can the idea have a lasting impact over time?
''',
)

In [103]:
scalability_reviewer = autogen.AssistantAgent(
    name="scalability_reviewer",
    llm_config=llm_config,
    system_message='''You are a critic. You review the idea from its Scalability aspects.
Growth Potential: Can the idea be scaled up if it’s successful?
Adaptability: Can the idea be adapted to different contexts or expanded in scope?
''',
)

In [104]:
risks_and_challenges_reviewer = autogen.AssistantAgent(
    name="risks_and_challenges_reviewer",
    llm_config=llm_config,
    system_message='''You are a critic. You review the idea from its risks and challenges aspects.
Risk Assessment: What are the potential risks associated with the idea, and how can they be mitigated?
Challenges: What obstacles might you face in implementing the idea, and how can they be overcome?
''',
)

In [105]:
Ethical_Considerations_reviewer = autogen.AssistantAgent(
    name="Ethical_Considerations_reviewer",
    llm_config=llm_config,
    system_message='''You are a critic. You review the idea from its Ethical aspects.
Ethical Impact: Does the idea adhere to ethical standards? Does it raise any ethical concerns?
Social Responsibility: Does the idea contribute positively to society?
''',
)

In [106]:
User_Acceptance_reviewer = autogen.AssistantAgent(
    name="User_Acceptance_reviewer",
    llm_config=llm_config,
    system_message='''You are a critic. You review the idea from its  User Acceptance aspects.
User Need: Does the idea address a real need or desire among the target audience?
Ease of Use: Will users find the idea easy to understand and use?
User Feedback: Is there positive feedback or interest from potential users?
''',
)

In [107]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

**Orchestrate the nested chats to solve the task**

In [108]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": Feasibility_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": innovation_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient":impact_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},

        {"recipient":scalability_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
        {"recipient":risks_and_challenges_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
        {"recipient":Ethical_Considerations_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},

            {"recipient":User_Acceptance_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},


     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]


In [109]:
Idea_modifier.register_nested_chats(
    review_chats,
    trigger=critic,
)

In [110]:
res = Idea_modifier.initiate_chat(
    recipient=critic,
    message=problem_idea,
    max_turns=2,
    summary_method="last_msg"
)

Idea_modifier (to Critic):

The problem is I want to make a girle to my girlefriend. 
 the idea is Instead of talking my self I use her friend to talk me on be half of me and makes her my girlefriend

--------------------------------------------------------------------------------
Critic (to Idea_modifier):

One significant weakness of this idea is that it lacks authenticity and honesty in building a relationship. Using a friend to speak on your behalf to make someone your girlfriend can create confusion, mistrust, and ultimately an unhealthy foundation for a relationship. A genuine connection is built on open and honest communication, mutual understanding, and shared experiences. Introducing a third party to mediate your feelings can distort the sincerity of your intentions and may lead to misunderstandings in the future. It could also potentially harm relationships with both the friend and the girlfriend if the truth comes to light, eroding trust and credibility in the process. Ultim